# SPARK ASSIGNMENT: NYC Parking Ticket Violation

In [1]:
#setting up environment variable

import os
import sys
os.environ["PYSPARK_PYTHON"] = "/opt/cloudera/parcels/Anaconda/bin/python"
os.environ["JAVA_HOME"] = "/usr/java/jdk1.8.0_232-cloudera/jre"
os.environ["SPARK_HOME"]="/opt/cloudera/parcels/SPARK2-2.3.0.cloudera2-1.cdh5.13.3.p0.316101/lib/spark2/"
os.environ["PYLIB"] = os.environ["SPARK_HOME"] + "/python/lib"
sys.path.insert(0, os.environ["PYLIB"] +"/py4j-0.10.6-src.zip")
sys.path.insert(0, os.environ["PYLIB"] +"/pyspark.zip")

In [2]:
#creating a spark session

from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('Ticket_Violation').master('local').getOrCreate()

spark

In [3]:
#Loading data into Spark Dataframe

s3path = 's3a://upgrad-data/Parking_Violation_Tickets.csv'
tickets = spark.read.format("csv").option("header", "true").load(s3path)

In [4]:
#Dataframe at a glance

tickets.show()

+--------------+--------+------------------+----------+----------+--------------+-----------------+------------+--------------+------------+------------+------------+-----------------------+------------------+------------------+---------------+-----------+--------------+------------+--------------+-------------------+----------------+---------------------------------+------------+--------------------+-------------------+-------------------+-----------+------------+--------------------+--------------------------+--------------------+------------------+-------------+---------------------+------------+------------+--------------+-------------------+---------------------+---------------------------------+-----------------+------------------------+
|Summons Number|Plate ID|Registration State|Plate Type|Issue Date|Violation Code|Vehicle Body Type|Vehicle Make|Issuing Agency|Street Code1|Street Code2|Street Code3|Vehicle Expiration Date|Violation Location|Violation Precinct|Issuer Precinct|Is

# Questions to Be Answered in the Analysis
## Examine the data
### 1.Find the total number of tickets for the year 2017.

In [5]:
#Importing all the Spark Sql functions

from pyspark.sql.functions import *

In [6]:
#Filtering the dataframe, keeping only entries made for 2017.

pattern = 'MM/dd/yyyy'

tickets2017=tickets.select('Registration State','Issue Date','Violation Code','Vehicle Body Type','Vehicle Make')\
.withColumn('Issue Date', unix_timestamp(tickets['Issue Date'], pattern).cast('timestamp')).filter(year('Issue Date')==2017)

tickets2017.show()

+------------------+-------------------+--------------+-----------------+------------+
|Registration State|         Issue Date|Violation Code|Vehicle Body Type|Vehicle Make|
+------------------+-------------------+--------------+-----------------+------------+
|                NY|2017-06-14 00:00:00|            47|             REFG|       MITSU|
|                NY|2017-06-13 00:00:00|             7|             SUBN|       ME/BE|
|                NY|2017-01-11 00:00:00|            78|             DELV|       FRUEH|
|                NY|2017-02-04 00:00:00|            40|             TAXI|       TOYOT|
|                NY|2017-01-26 00:00:00|            64|              VAN|       INTER|
|                NY|2017-04-30 00:00:00|            20|             SUBN|       DODGE|
|                FL|2017-02-03 00:00:00|            36|               4D|         BMW|
|                NY|2017-05-27 00:00:00|            38|              VAN|       CHEVR|
|                NY|2017-05-31 00:00:00|   

In [7]:
#Assuming there are no null values in the dataframe.
#Using count method to calculate the total number of tickets for the year 2017

tickets2017.count()

5431918

Total number of tickets for the year 2017 is 5431918

### 2.Find out the number of unique states from where the cars that got parking tickets came.

In [8]:
#Using select and distinct to find the unique 'Registartion State' entries.

tickets2017.select('Registration State').distinct().count()

65

There are 65 distinct values of Registration State.

*There is a numeric entry '99' in the column, which should be corrected. Replace it with the state having the maximum entries. Provide the number of unique states again.

In [9]:
#Finding the state having maximum entry.

tickets2017.groupBy('Registration State').count().orderBy('count', ascending = False).show()

+------------------+-------+
|Registration State|  count|
+------------------+-------+
|                NY|4273951|
|                NJ| 475825|
|                PA| 140286|
|                CT|  70403|
|                FL|  69468|
|                IN|  45525|
|                MA|  38941|
|                VA|  34367|
|                MD|  30213|
|                NC|  27152|
|                TX|  18827|
|                IL|  18666|
|                GA|  17537|
|                99|  16055|
|                AZ|  12379|
|                OH|  12281|
|                CA|  12153|
|                ME|  10806|
|                SC|  10395|
|                MN|  10083|
+------------------+-------+
only showing top 20 rows



In [10]:
#Since NY has the maximum entry, replacing '99' with NY state and counting number of unique states again.

tickets2017.select('Registration State').withColumn('Registration State',when(tickets2017["Registration State"]=="99",lit('NY'))\
                       .otherwise(tickets2017["Registration State"])).distinct().count()

64

There are 64 distinct values of Registration State after replacing '99' by NY

### 3.Display the top 20 states with the most number of tickets along with their ticket count.

In [11]:
#Displaying top 20 states with the most number of tickets along with their ticket count.

tickets2017.groupBy('Registration State').count().orderBy('count', ascending = False).show()

+------------------+-------+
|Registration State|  count|
+------------------+-------+
|                NY|4273951|
|                NJ| 475825|
|                PA| 140286|
|                CT|  70403|
|                FL|  69468|
|                IN|  45525|
|                MA|  38941|
|                VA|  34367|
|                MD|  30213|
|                NC|  27152|
|                TX|  18827|
|                IL|  18666|
|                GA|  17537|
|                99|  16055|
|                AZ|  12379|
|                OH|  12281|
|                CA|  12153|
|                ME|  10806|
|                SC|  10395|
|                MN|  10083|
+------------------+-------+
only showing top 20 rows



## Aggregation Tasks

### 1.How often does each violation code occur? Display the frequency of the top five violation codes.

In [12]:
#Displaying the frequency of top five violation codes.

tickets2017.groupBy('Violation Code').count().orderBy('count', ascending = False).show(5)

+--------------+------+
|Violation Code| count|
+--------------+------+
|            21|768087|
|            36|662765|
|            38|542079|
|            14|476664|
|            20|319646|
+--------------+------+
only showing top 5 rows



The top five violations codes are 21, 36, 38, 14 and 20

### 2.How often does each 'vehicle body type' get a parking ticket? How about the 'vehicle make'? Find the top 5 for both.

In [13]:
#Displaying frequency of parking tickets for each 'Vehicle Body Type'

tickets2017.groupBy('Vehicle Body Type').count().orderBy('count', ascending = False).show(5)

+-----------------+-------+
|Vehicle Body Type|  count|
+-----------------+-------+
|             SUBN|1883954|
|             4DSD|1547312|
|              VAN| 724029|
|             DELV| 358984|
|              SDN| 194197|
+-----------------+-------+
only showing top 5 rows



The top five vehicle body type that often get a parking ticket are SUBN, 4DSD, VAN, DELV and SDN

In [14]:
#Displaying frequency of parking tickets for each 'Vehicle Make'

tickets2017.groupBy('Vehicle Make').count().orderBy('count', ascending = False).show(5)

+------------+------+
|Vehicle Make| count|
+------------+------+
|        FORD|636844|
|       TOYOT|605291|
|       HONDA|538884|
|       NISSA|462017|
|       CHEVR|356032|
+------------+------+
only showing top 5 rows



The top five vehicle bmake that often get a parking ticket are FORD, TOYOT, HONDA,NISSA and CHEVR

### 3.Let’s try and find some seasonality in this data:

a.First, divide the year into 4 seasons, and find the frequencies of tickets for each season.

In [15]:
#Creating a UDF to divide the year into four seasons as per the following code:
# Month 1, 2 & 12 - winter
# Month 3,4 & 5 - Spring
# Month 6, 7 & 8 - Summer
# Month 9,10 & 11 - Autumn

add_season = udf(lambda date: 'Summer' if date.month in [6, 7, 8] else ('Winter' if date.month in [1, 2, 12] else ('Spring' if date.month in [3, 4, 5] else 'Autumn')))

#Displaying the season categorization

tickets_seasonWise=tickets2017.withColumn('Season', add_season('Issue Date')).select('Registration State', 'Violation Code', 'Season')

tickets_seasonWise.show()


+------------------+--------------+------+
|Registration State|Violation Code|Season|
+------------------+--------------+------+
|                NY|            47|Summer|
|                NY|             7|Summer|
|                NY|            78|Winter|
|                NY|            40|Winter|
|                NY|            64|Winter|
|                NY|            20|Spring|
|                FL|            36|Winter|
|                NY|            38|Spring|
|                NY|            14|Spring|
|                NY|            75|Spring|
|                NY|            10|Spring|
|                NY|            69|Summer|
|                NJ|            21|Winter|
|                NY|            38|Spring|
|                NY|            48|Winter|
|                NJ|            21|Winter|
|                VA|            21|Summer|
|                NJ|            68|Winter|
|                NY|            51|Winter|
|                NY|             9|Summer|
+----------

In [16]:
#Displaying the frequencies of tickets for each season using UDF.

season_freq=tickets_seasonWise.select('Season').groupBy('Season').count()

season_freq.show()

+------+-------+
|Season|  count|
+------+-------+
|Spring|2873383|
|Summer| 852866|
|Autumn|    979|
|Winter|1704690|
+------+-------+



The maximum violations are made in Spring season while the least violations are made in Autumn season.

b.Then, find the three most common violations for each of these seasons. 

In [17]:
# Displaying three most common violations for Spring

tickets_seasonWise.filter(tickets_seasonWise['Season']=='Spring').select('Violation Code').groupBy('Violation Code')\
.count().orderBy('count', ascending=False).show(3)

+--------------+------+
|Violation Code| count|
+--------------+------+
|            21|402424|
|            36|344834|
|            38|271167|
+--------------+------+
only showing top 3 rows



Most common violations made in Spring are 21, 36 and 38.

In [18]:
# Displaying three most common violations for Summer

tickets_seasonWise.filter(tickets_seasonWise['Season']=='Summer').select('Violation Code').groupBy('Violation Code')\
.count().orderBy('count', ascending=False).show(3)

+--------------+------+
|Violation Code| count|
+--------------+------+
|            21|127352|
|            36| 96663|
|            38| 83518|
+--------------+------+
only showing top 3 rows



Most common violations made in Summer are 21, 36 and 38.

In [19]:
# Displaying three most common violations for Winter

tickets_seasonWise.filter(tickets_seasonWise['Season']=='Winter').select('Violation Code').groupBy('Violation Code')\
.count().orderBy('count', ascending=False).show(3)

+--------------+------+
|Violation Code| count|
+--------------+------+
|            21|238183|
|            36|221268|
|            38|187386|
+--------------+------+
only showing top 3 rows



Most common violations made in Winter are 21, 36 and 38.

In [20]:
# Displaying three most common violations for Autumn

tickets_seasonWise.filter(tickets_seasonWise['Season']=='Autumn').select('Violation Code').groupBy('Violation Code')\
.count().orderBy('count', ascending=False).show(3)

+--------------+-----+
|Violation Code|count|
+--------------+-----+
|            46|  231|
|            21|  128|
|            40|  116|
+--------------+-----+
only showing top 3 rows



Contrary to other seasons most common violations made in Autumn are 46, 21 and 40.

### 4.The fines collected from all the instances of parking violation constitute a source of revenue for the NYC Police Department. Let’s take an example of estimating this for the three most commonly occurring codes:

a.Find the total occurrences of the three most common violation codes.

In [21]:
#Total occurrences of the three most common 'Violation Codes'.

common_violations=tickets2017.select('Violation Code').groupBy('Violation Code').count().orderBy('count', ascending=False)

common_violations.show(3)

+--------------+------+
|Violation Code| count|
+--------------+------+
|            21|768087|
|            36|662765|
|            38|542079|
+--------------+------+
only showing top 3 rows



The three most common 'Violation Codes' are 21, 36 and 38.



Then, visit the website:
http://www1.nyc.gov/site/finance/vehicles/services-violation-codes.page
It lists the fines associated with different violation codes. They’re divided into two categories: one for the highest-density locations in the city and the other for the rest of the city. For the sake of simplicity, take the average of the two.

b.Using this information, find the total amount collected for each of the three violation codes with the maximum tickets. State the code that has the highest total collection (only based on the top 3 tickets).

In [22]:
#Average of fines as per the link given:

# violation code   fine avg
#    21              65
#    36              50
#    38              50

#Adding Avg_fine column to common_violations dataframe.

from pyspark.sql.functions import lit

violations_avgFine=common_violations.withColumn('Avg_fine',when(common_violations['Violation Code'] ==21,lit(65))\
                         .when(common_violations['Violation Code'] ==36, lit(50))\
                         .when(common_violations['Violation Code'] ==38, lit(50))\
                         .otherwise('null'))

violations_avgFine.show(3)

+--------------+------+--------+
|Violation Code| count|Avg_fine|
+--------------+------+--------+
|            21|768087|      65|
|            36|662765|      50|
|            38|542079|      50|
+--------------+------+--------+
only showing top 3 rows



The average fine for violation codes 21, 36 and 38 are 65, 50 and 50 respectively.

In [23]:
#Displaying total fine collected for most common 'Vioaltion Code'

violations_totalFine=violations_avgFine.withColumn('Fine Collected',violations_avgFine['count']*violations_avgFine['Avg_fine'])\
                                          .orderBy('Fine Collected', ascending=False)

violations_totalFine.show(3)

+--------------+------+--------+--------------+
|Violation Code| count|Avg_fine|Fine Collected|
+--------------+------+--------+--------------+
|            21|768087|      65|   4.9925655E7|
|            36|662765|      50|    3.313825E7|
|            38|542079|      50|    2.710395E7|
+--------------+------+--------+--------------+
only showing top 3 rows



The code with highest total fine collection is 21

c.Find the top 3 states that have the highest ticket revenue based on the top 3 violation codes alone. (Hint: Use the column 'Registration State'.)

In [24]:
from pyspark.sql.types import IntegerType

fine_for_each_Code=udf(lambda code: 65 if code==21 else (50), IntegerType() )


tickets_seasonWise.select('Violation Code','Registration State').filter(tickets_seasonWise['Violation Code']\
                  .isin(21,36,38)).withColumn('Fine',fine_for_each_Code('Violation Code')).groupBy('Registration State')\
                  .sum('Fine').orderBy('sum(Fine)', ascending=False).show(3)

+------------------+---------+
|Registration State|sum(Fine)|
+------------------+---------+
|                NY| 79015600|
|                NJ|  6571250|
|                PA|  2969650|
+------------------+---------+
only showing top 3 rows



The top three states with highest ticket revenue are NY, NJ and PA.

### 4.What can you intuitively infer from these findings?

Based on the analysis, we found the following facts:

1.Total number of tickets for the year 2017 is 5431918 <br>
2.There are 64 distinct values of Registration State. <br>
3.The three most common 'Violation Codes' are 21, 36 and 38.<br>
4.The code with highest total fine collection is 21. <br>
5.The top three states with highest ticket revenue are NY, NJ and PA. <br>

Observations:

1.It can be observed that a huge chunk of violations were made from January to June i.e Winter and Spring season. <br>
2.We can also observe that there is a very sharp decline in the violations from July onwards i.e Summer and Autumn season. Autumn being the season with least number of violations. <br>
3.Highest violation &collection was by Code-21(No parking where parking is not allowed by sign, street marking or traffic control device.) <br>
<br>
We can infer that in winter season due to haze and fog people tend to not see the 'NO' parking signs and end up getting tickets for the violations.

We can also infer that fog/haze affect New York state the most.

From all the observations we can also infer that people from New York and New Jersey don't have parking etiquettes as the maximum violations are parking violations (21,38,46 and 40) as opposed to other violations.